In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 758.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 31.3 MB/s eta 0:00:00


In [3]:
import pdfplumber
import pandas as pd
import re

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to find the required information using regex and capture surrounding text
def find_information(text):
    info = {}
    surrounding_context = {}

    # Extracting information using regex patterns
    contract_number_match = re.search(r'Contract Number:\s*(.*)', text)
    if contract_number_match:
        info['contract_number'] = contract_number_match.group(1)
    else:
        info['contract_number'] = "Not found"
    if re.search(r'entered into as of\s*(\w+ \d+, \d+)', text):
        info['start_date'] = re.search(r'entered into as of\s*(\w+ \d+, \d+)', text).group(1)
    else:
      info['start_date'] = "Not found"
    end_date_match = re.search(r'terminating on\s*(\w+ \d+, \d+)', text)
    if end_date_match:
        info['end_date'] = end_date_match.group(1)
    else:
        end_date_start = text.find("remain in effect until")
        surrounding_context['end_date_context'] = text[end_date_start:end_date_start + 500]
        info['end_date'] = "Not found"

    supplier_name_match = re.search(r'Party B:[\s\S]+Name:\s*(.*)', text)
    if supplier_name_match:
        info['supplier_name'] = supplier_name_match.group(1)
    else:
        info['supplier_name'] = "Not found"

    invoice_id_match = re.search(r'unique invoice number, starting with\s*(\w+-\d+)', text)
    if invoice_id_match:
        info['invoice_id'] = invoice_id_match.group(1)
    else:
        info['invoice_id'] = "Not found"

    amount_of_contract_match = re.search(r'Total Contract Amount:\s*\$(.*)', text)
    if amount_of_contract_match:
        info['amount_of_contract'] = amount_of_contract_match.group(1)
    else:
        amount_start = text.find("Total Contract Amount:")
        surrounding_context['amount_of_contract_context'] = text[amount_start:amount_start + 500]
        info['amount_of_contract'] = "Not found"

    # Extracting KPIs
    kpi_pattern = re.compile(r'KPIs:[\s\S]+?(?=Reporting Requirements|Miscellaneous)')
    kpis = kpi_pattern.search(text)
    if kpis:
        info['KPIs'] = kpis.group(0).strip()
    else:
        info['KPIs'] = "Not found"

    # Extracting Reporting Requirements
    reporting_pattern = re.compile(r'Reporting Requirements:[\s\S]+?(?=Miscellaneous)')
    reporting_requirements = reporting_pattern.search(text)
    if reporting_requirements:
        info['reporting_requirements'] = reporting_requirements.group(0).strip()
    else:
        info['reporting_requirements'] = "Not found"

    # Extracting Scope of Services
    scope_pattern = re.compile(r'Scope\s*of\s*Services\s*:[\s\S]+?(?=Term\s*and\s*Termination|$)')
    scope_of_services_match = scope_pattern.search(text)
    if scope_of_services_match:
        scope_of_services_text = scope_of_services_match.group(0).strip()

        # Extract subheadings and bullet points
        subheadings_pattern = re.compile(r'([A-Z][\w\s]*):\s*([\s\S]+?)(?=[A-Z][\w\s]*:|$)')
        subheadings = subheadings_pattern.findall(scope_of_services_text)

        subheadings_dict = {}
        for subheading in subheadings:
            subheadings_dict[subheading[0]] = subheading[1].strip().split('\n')

        info['scope_of_services'] = subheadings_dict
    else:
        scope_start = text.find("Scope of Services")
        surrounding_context['scope_of_services_context'] = text[scope_start:scope_start + 1000]
        info['scope_of_services'] = "Not found"

    return info, surrounding_context

# Function to save the extracted information to an Excel file
def save_to_excel(info, surrounding_context, excel_path):
    data = {**info, **surrounding_context}
    df = pd.DataFrame([data])
    df.to_excel(excel_path, index=False)

# Main execution
pdf_path = '/content/Sample Contract.pdf'
excel_path = '/content/Save_info.xlsx'

text = extract_text_from_pdf(pdf_path)
info, surrounding_context = find_information(text)
save_to_excel(info, surrounding_context, excel_path)

excel_path

'/content/Save_info.xlsx'

In [4]:
pdf_path = '/content/Sample Contract.pdf'
excel_path = '/content/Save_info.xlsx'

# Extract text from PDF
text = extract_text_from_pdf(pdf_path)

# Extract information from the text using regex
info, surrounding_context = find_information(text)

# Print specific information
print("Extracted Information:")
print(f"Contract Number: {info['contract_number']}")
print(f"Start Date: {info['start_date']}")
print(f"End Date: {info['end_date']}")
print(f"Supplier Name: {info['supplier_name']}")
print(f"Invoice ID: {info['invoice_id']}")
print(f"Amount of Contract: {info['amount_of_contract']}")
print(f"KPIs:\n{info['KPIs']}")
print(f"Reporting Requirements:\n{info['reporting_requirements']}")
print(f"Scope of Services:\n{info['scope_of_services']}")

# Save extracted information to Excel
save_to_excel(info, surrounding_context, excel_path)

# Display the path to the saved Excel file
print("Excel file saved at:", excel_path)


Extracted Information:
Contract Number: XYZ-ABC-2024-01
Start Date: July 1, 2024
End Date: July 1, 2026
Supplier Name: Jane Smith, ABC Cleaning Services
Invoice ID: INV-001
Amount of Contract: Not found
KPIs:
KPIs:
● Uptime: Percentage of scheduled cleaning services delivered on time. Target: 99%.
● Response Time: Average time taken to respond to additional cleaning requests. Target:
2 hours.
● Customer Satisfaction: Feedback scores from XYZ Corporation's staff. Target: 4.5 out
of 5.
● Incident Frequency: Number of complaints or issues reported per month. Target: Less
than 2 per month.
10.2.
Reporting Requirements:
Reporting Requirements:
● Monthly Performance Reports: Detailed metrics and analysis of KPIs, including service
delivery times, response times, and customer feedback.
● Incident Reports: Logs detailing any reported incidents, including descriptions, dates,
and resolutions.
● Maintenance Activity Reports: Summary of all cleaning activities conducted, including
dates, times, a